In [1]:
# import os.path as osp

from tqdm import tqdm
import numpy as np
import pandas as pd
import vaex as vpd
from numba import jit
import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.utils.loop import remove_self_loops

In [2]:
w_df = vpd.open('../data/external/wboson.hdf5').to_pandas_df().set_index('evt_num')
# q_df = vpd.open('../data/external/qstar.hdf5')

In [3]:
@jit
def get_edges(num_nodes):
    nodes = np.arange(num_nodes)
    edge_idx = np.vstack((
        np.repeat(nodes, num_nodes),
        np.repeat(nodes, num_nodes).reshape(-1, num_nodes).T.flatten()
    ))
    return edge_idx

In [ ]:
data = []
num_evts = w_df.index.max()
mmtm_slc = [slice(None), slice(0, 4)]

for evt_num in tqdm(range(0, num_evts)):
    mmtm = torch.from_numpy( # create torch tensor
        w_df.loc[evt_num, 'px':'E'].values # get numpy values for 4-momentum
    )
    num_nodes = mmtm.shape[0]
    edge_idx = torch.tensor(get_edges(num_nodes), dtype=torch.long)
    edge_idx, _ = remove_self_loops(edge_idx)
    graph = Data(x=mmtm, edge_index=edge_idx)
    data.append(graph)

 10%|█         | 10202/99999 [16:26<3:12:28,  7.78it/s]